 # Table of Contents
<div class="toc" style="margin-top: 1em;"><ul class="toc-item" id="toc-level0"><li><span><a href="http://0.0.0.0:8888/notebooks/hermite/discrete_gradient.ipynb#Discrete-gradient-operators" data-toc-modified-id="Discrete-gradient-operators-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Discrete gradient operators</a></span></li><li><span><a href="http://0.0.0.0:8888/notebooks/hermite/discrete_gradient.ipynb#Gradient-of-a-piecewise-linear-function" data-toc-modified-id="Gradient-of-a-piecewise-linear-function-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Gradient of a piecewise linear function</a></span><ul class="toc-item"><li><span><a href="http://0.0.0.0:8888/notebooks/hermite/discrete_gradient.ipynb#A-detour-using-projections" data-toc-modified-id="A-detour-using-projections-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>A detour using projections</a></span><ul class="toc-item"><li><span><a href="http://0.0.0.0:8888/notebooks/hermite/discrete_gradient.ipynb#How-does-the-assembly-of-the-projection-problem-work?" data-toc-modified-id="How-does-the-assembly-of-the-projection-problem-work?-2.1.1"><span class="toc-item-num">2.1.1&nbsp;&nbsp;</span>How does the assembly of the projection problem work?</a></span></li></ul></li><li><span><a href="http://0.0.0.0:8888/notebooks/hermite/discrete_gradient.ipynb#And-back" data-toc-modified-id="And-back-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>And back</a></span></li><li><span><a href="http://0.0.0.0:8888/notebooks/hermite/discrete_gradient.ipynb#Using-the-PETSc-backend" data-toc-modified-id="Using-the-PETSc-backend-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Using the PETSc backend</a></span></li></ul></li><li><span><a href="http://0.0.0.0:8888/notebooks/hermite/discrete_gradient.ipynb#Discrete-gradient-for-DKT" data-toc-modified-id="Discrete-gradient-for-DKT-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Discrete gradient for DKT</a></span><ul class="toc-item"><li><span><a href="http://0.0.0.0:8888/notebooks/hermite/discrete_gradient.ipynb#Tests" data-toc-modified-id="Tests-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Tests</a></span></li><li><span><a href="http://0.0.0.0:8888/notebooks/hermite/discrete_gradient.ipynb#Iterating-over-facets" data-toc-modified-id="Iterating-over-facets-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Iterating over facets</a></span></li><li><span><a href="http://0.0.0.0:8888/notebooks/hermite/discrete_gradient.ipynb#C++-implementation" data-toc-modified-id="C++-implementation-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>C++ implementation</a></span></li><li><span><a href="http://0.0.0.0:8888/notebooks/hermite/discrete_gradient.ipynb#Tests" data-toc-modified-id="Tests-3.4"><span class="toc-item-num">3.4&nbsp;&nbsp;</span>Tests</a></span></li></ul></li><li><span><a href="http://0.0.0.0:8888/notebooks/hermite/discrete_gradient.ipynb#Dump" data-toc-modified-id="Dump-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Dump</a></span><ul class="toc-item"><li><span><a href="http://0.0.0.0:8888/notebooks/hermite/discrete_gradient.ipynb#A-quick-retesting-of-the-assembly-of-the-matrix-for-$\nabla_h$" data-toc-modified-id="A-quick-retesting-of-the-assembly-of-the-matrix-for-$\nabla_h$-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>A quick retesting of the assembly of the matrix for $\nabla_h$</a></span></li></ul></li></ul></div>

# Discrete gradient operators

This notebook collects my progress towards the definition of the operator

$$\nabla_h:W_h \rightarrow \Theta_h$$

as defined in (Bartels, 2015) §8.2.1. We proceed as follows:

1. Computation of (global) exact gradient for CG1 functions, i.e. as an operator from CG1 into CG0. This serves to test the idea with a simpler operator. See [Gradient of a piecewise linear function](#Gradient-of-a-piecewise-linear-function).
2. Computation of (global) gradient $\nabla_h$ for DKT functions, i.e. as an operator from DKT into CG2^2. See [Discrete gradient for DKT](#Discrete-gradient-for-DKT). This implementation basically works, but being pure python is extremely slow. A small improvement might be achieved by [Iterating over facets](#Iterating-over-facets). However we are building a huge matrix and multiplying the system matrix by it and its transpose, which is very slow.
3. Implementation in C++ of the assembly process. This requires a new assembler for each problem or at least for each family of related ones, but should be pretty straightfoward if one uses FEniCS compiled forms.

In [ ]:
## The usual boilerplate...

from itertools import chain
from dolfin import *
from petsc4py import PETSc
import numpy as np
import matplotlib.pyplot as pl
%matplotlib inline

np.set_printoptions(precision=3, linewidth=150)

def __nbinit__():
    global __all__
    __all__ = ['compute_discrete_gradient_dense', 
               'compute_discrete_gradient',
               'compute_dkt_gradient']
    global parameters
    # instruct FFC to produce the code for evaluate_basis_derivatives()
    parameters["form_compiler"]["no-evaluate_basis_derivatives"] = False

__nbinit__()

# Gradient of a piecewise linear function

Let $u_{h}$ be a piecewise linear function over a mesh $\mathcal{T}_{h}$, that is $u_{h} \in V_{h} =\operatorname{span} \{ \phi^i_{h} \}$, where $V_{h}$ is a CG1 finite element space and $\phi^i$ are locally in $P_{1} (T)$ for some simplex $T \in \mathcal{T}_{h}$. Then $u_{h}'$ is piecewise constant, i.e. $u'_{h} \in V_{h}' =\operatorname{span} \{ \psi _{h}^i \}$, a DG0 space over $\mathcal{T}_{h}$, where $\psi^i_{h}$ are constant over simplices. Denote by $U = (U_{i})$ and $U' = (U'_{i})$ their vectors of coefficients wrt. the standard bases of nodal functions over $V_{h}$ and $V_{h}'$ respectively. We want to compute a matrix $M$ for the gradient operator

\\[ \nabla _{h} : V_{h} \rightarrow V_{h}' \\]

such that $(M U)_{j} = U'_{j}$.
Since $u'_{h}$ is not in the same space as $u_{h}$, one way of computing the coefficients $U'$ in FEniCS is to `df.project(df.grad(u), V)` as mentioned above. But here we manually compute the transformation without solving a linear system. Note that if we were only interested in evaluation of $u_{h}'$, because $u_{h} = \sum_{i} U_{i} \phi^i_{h}$ and by linearity $u_{h}' = \sum_{i} U_{i} (\phi_{h}^i)'$. We could then use `df.evaluate_basis_derivatives()` to evaluate the $\phi _{i}'$ and we would have finished.

But again, we are interested in obtaining the new coefficients $U_{i}'$ such that
\\[ u' = \sum_{i} U'_{i} \psi _{i} . \\]

We assemble $M$ cell by cell as follows, assuming that both $V$ and $V'$ are defined over the same mesh $\mathcal{T}_{h}$ with $N$ cells.


  * First, for any given cell $T$ compute the mapping $L_{T}$ of the dofs in $V_{|T}$ to the dofs in $V'_{|T}$.
  * Then apply this mapping to the components of $U_{|T}$ to obtain the coefficients $U'_{|T}$.

To explain the idea, consider first a 1D mesh for simplicity and assume that we have an index $i \in [N]$ over the cells of the mesh such that:

  * $U_{i}, U_{i + 1}$ are the coefficients for the dofs in $V$ over cell $i$.
  * $U_{i}'$ is the coefficient for the only dof in $V'$ (the constant 1) over cell $i$.

This assumption is actually a description of the mapping $L_{T}$ mentioned above for each interval. Then we only need to compute $(\phi^i_{h})'$ (this is not a typo, we are actually computing the derivative) for each $i$ to obtain the new vector of coefficients $U_{|T}'$:

\\[ M_{T} U_{|T} = \left(\begin{array}{ccccc}
     \phi _{h}^{0\prime} & \phi _{h}^{1\prime} &  &  &  \\
     & \phi _{h}^{1\prime} & \phi _{h}^{2\prime} &  &  \\
     &  & \phi _{h}^{2\prime} & \phi _{h}^{3\prime} &  \\
     &  &  & \phi _{h}^{3\prime} & \phi _{h}^{4\prime}
   \end{array}\right) \left(\begin{array}{c}
     U_{0}\\
     U_{1}\\
     U_{2}\\
     U_{3}\\
     U_{4}
   \end{array}\right) = \left(\begin{array}{c}
     U_{0}'\\
     U_{1}'\\
     U_{2}'\\
     U_{3}'
   \end{array}\right) . \\]
   
Note that in this particular instance and with the assumptions made, we obtain a band matrix and computing $U'$ is actually a matter of computing the dot product of a subset of the vector of constants $(\phi _{h}^{0\prime}, ..., \phi _{h}^{3\prime})$ with $U$.

However in other situations, e.g. in higher dimensions, the ordering of the nodes will not be as convenient. We proceed then more generally as follows, now for the global matrix. Let $i$ run over all cell indices and let $\iota _{i}$ be its local-to-global mapping for $V$ and $\iota'_{i}$ for $V'$. Initialise the global matrix $M$ of the discrete gradient to zero. At each step of the construction of $M$ we edit row $r = \iota _{i}' (0)$, i.e. the row whose product by $U$ will return the coefficient for the dof in $V'$ corresponding to cell $i$.

Now, for every vertex $z_{j}, j \in [d]$ in cell $i$ compute the value of the derivative of the global dof $\phi _{h}^{\iota _{r} (j)}$ and set check this, I'm not sure after so many changes in notation

\\[ M_{rj} = \phi _{h}^{\iota _{r} (j)\prime} (z_{j}), j \in [d] . \\]

Here is a straightforward implementation of this procedure using AIJ sparse matrices in PETSc

In [ ]:
def discrete_gradient_operator_dense(V, Vp):
    """ Build dense matrix of the discrete gradient
    operator between V and Vp.
    
    Arguments
    ---------
        V: Source FunctionSpace of type CG1.
        Vp: Target (Vector)FunctionSpace of type DG0.
        
    Returns
    -------
        Dh: np.ndarray of shape (Vp.dim(), V.dim()) such that,
            for u in V:
        
                np.dot(Dh, u.vector().array())
              
            yields the vector of coefficients of the discrete
            gradient of u in Vp.
    """
    gdim = V.mesh().geometry().dim()
    assert V.ufl_element().shortstr()[:3] == 'CG1', "Need a CG1 source space"
    assert Vp.ufl_element().shortstr()[:14] == 'Vector<%d x DG0' % gdim or\
           Vp.ufl_element().shortstr()[:3] == 'DG0',\
           "I need a DG0 (Vector)FunctionSpace of the right number of components"
    assert Vp.mesh().num_cells() * gdim == Vp.dim(),\
           "FunctionSpace dimensions don't match."
    assert Vp.mesh().num_cells() == V.mesh().num_cells(),\
           "Meshes don't match"

    dm = V.dofmap()
    dmp = Vp.dofmap()
    e = V.element()
    coords = V.tabulate_dof_coordinates().reshape((-1, gdim))
    coordsp = Vp.tabulate_dof_coordinates().reshape((-1, gdim))

    Dh = np.zeros((Vp.dim(), V.dim()))
    warning("Assuming all cells have the same number of dofs")
    vals = np.zeros(dm.num_element_dofs(0)*gdim)
    for cell_id in range(Vp.mesh().num_cells()):
        rows = dmp.cell_dofs(cell_id)
        columns = dm.cell_dofs(cell_id)
        dof_coords = coords[columns,:]
        point = dof_coords[0] # Any point in the (closure of the) cell will do.
        e.evaluate_basis_derivatives_all(1, vals, point, dof_coords, 1)
        #print("rows=%s, columns=%s, point=%s, dof_coords=%s, vals=%s" %
        #     (rows, columns, point, dof_coords, vals.round(2)))
        # Reshape [df_1/dx, df_1/dy, df_2/dx, df_2/dy, ...] into
        # [[df_1/dx, df_2/dx, ...],[df_1/dy, df_2/dy, ...]] :
        #Dh[FIXTHIS:"rows, columns"] = vals.reshape((-1, gdim)).T.copy()
        for offset in range(gdim):
            Dh[rows[offset], columns] = vals[offset::gdim].copy()

    return Dh

def compute_discrete_gradient_dense(V, Vp, u):
    """ Returns the discrete gradient of u.
    NOTE: this implementation uses dense matrices internally.

    Arguments
    ---------
        V: Source FunctionSpace of type CG1. (redundant...)
        Vp: Target (Vector)FunctionSpace of type DG0.

    Returns
    -------
        A Function in Vp interpolating the gradient of u.
    """

    Dh = discrete_gradient_operator_dense(V, Vp)
    up = Function(Vp)
    up.vector().set_local(np.dot(Dh, u.vector().array()))
    up.vector().apply('insert')

    return up

In order to test the previous code we start with an expression whose derivative we can compute:

In [ ]:
LEFT, RIGHT = -pi, 2*pi
mesh = IntervalMesh(100, LEFT, RIGHT)
V = FunctionSpace(mesh, "CG", 1)
Vp = FunctionSpace(mesh, "DG", 0)

#x = SpatialCoordinate(mesh)
#f = sin(x[0])
#u = project(f, V)
u = interpolate(Expression("sin(x[0])", element=V.ufl_element()), V)
dg_u = compute_discrete_gradient_dense(V, Vp, u)

# Test
#up = project(f.dx(0), Vp)
up = interpolate(Expression("cos(x[0])", element=Vp.ufl_element()), Vp)
print("Error: %e" % (dg_u.vector() - up.vector()).norm('linf'))
plot(up)
plot(dg_u)
_ = pl.xlim((LEFT*1.1, RIGHT*1.1)), pl.ylim((-1.1,1.1))

## A detour using projections

The following method computes the solution via a projection onto DG0 but instead of letting FEniCS assemble the mass matrix, we make use of the fact that it is diagonal with entries given by the cell volume. Then it is trivial to invert it and multiply the rhs of the projection problem. The idea and code are taken from [this question on FEniCS Q&A](https://fenicsproject.org/qa/12634/gradient-of-a-cg-order-one-element?show=12646#a12646).

In [ ]:
mesh = UnitCubeMesh(10, 10, 10)
W = FunctionSpace(mesh, 'CG', 1)
Q = VectorFunctionSpace(mesh, 'DG', 0)
p, q = TrialFunction(Q), TestFunction(Q)

f = Function(W)
f_vec = f.vector()
f_vec.set_local(np.random.rand(f_vec.local_size()))
f_vec.apply('insert')

We first compute the gradient via a projection. **NOTE** that the computation for the form $L$ will have to interpolate $f$, which is in "W", into the space $T$ where the test function $q$ lives in order to compute the integral. [WHY?]

In [ ]:
%%time

L = inner(nabla_grad(f), q)*dx
M = assemble(inner(p, q)*dx)
b = assemble(L)

grad_f0 = Function(Q)
x0 = grad_f0.vector()
_ = solve(M, x0, b)

MiroK's idea (which amounts to building our matrix $D_h$) in the forum is to "assemble the action of Minv onto the rhs - no mass matrix needed". The improvement in speed is huge:

In [ ]:
%%time

MinvL = (1/CellVolume(Q.mesh()))*inner(nabla_grad(f), q)*dx
x = assemble(MinvL)

In [ ]:
grad_f = Function(Q, x)
print("Error: %e" % (x - x0).norm('linf'))

Note that in the first approach we compute $M$, then solve $M x = b$ and in the second we compute $M^{-1}$ directly to obtain $x = M^{-1} b$.

### How does the assembly of the projection problem work?

Write the compiled form to a file and check `dgrad_cell_integral_0_otherwise::tabulate_tensor()` for the implementation. Recall that `u.dx()` in the form is the derivative of a terminal node in UFL so it is the responsibility of FFC to fill that. How does this work again?

In [ ]:
import ffc
with open("/tmp/dgrad.h", "wt") as fd:
    out = ffc.compile_form(MinvL, prefix="dgrad")
    fd.write(out[0])

## And back

We can use our function instead. This will be much slower and also use a lot of memory because of the dense matrix we are using. The error is different, though. **Could this be significant?**

In [ ]:
%%time

dg_f = compute_discrete_gradient_dense(W, Q, f)

In [ ]:
print("Error: %e" % (dg_f.vector() - grad_f0.vector()).norm('linf'))

## Using the PETSc backend

By using PETSc We can achieve a (back-of-the-napkin) speed improvement of up to 50 times wrt. the implementation with dense matrices and stay in the same range of running times than `project()`. We can only test for manageable matrix sizes but we can now of course handle bigger matrices. The speedup depends greatly on the size of the matrices.

In [ ]:
def discrete_gradient_operator(V, Vp):
    """ Build sparse PETSc matrix of the discrete gradient
    operator between V and Vp.
    
    Arguments
    ---------
        V: Source FunctionSpace of type CG1.
        Vp: Target (Vector)FunctionSpace of type DG0.
        
    Returns
    -------
        Dh: PETSc sparse matrix such that for u in V,
              Dh * u.vector().array()
            yields the vector of coefficients of the discrete
            gradient of u in Vp.
    """   
    gdim = V.mesh().geometry().dim()
    assert V.ufl_element().shortstr()[:3] == 'CG1',\
           "Need a CG1 source space"
    assert Vp.ufl_element().shortstr()[:14] == 'Vector<%d x DG0' % gdim or\
           Vp.ufl_element().shortstr()[:3] == 'DG0',\
           "I need a DG0 (Vector)FunctionSpace of the right number of components"
    assert Vp.mesh().num_cells() * gdim == Vp.dim(),\
           "FunctionSpace dimensions don't match"
    assert Vp.mesh().num_cells() == V.mesh().num_cells(),\
           "Number of cells in the meshes don't match"

    dm = V.dofmap()
    dmp = Vp.dofmap()
    e = V.element()
    coords = V.tabulate_dof_coordinates().reshape((-1, gdim))
    coordsp = Vp.tabulate_dof_coordinates().reshape((-1, gdim))

    Dh = PETSc.Mat()
    Dh.create(PETSc.COMM_WORLD)
    Dh.setSizes([Vp.dim(), V.dim()])
    Dh.setType("aij")
    Dh.setUp()
    warning("Assuming all cells have the same number of dofs")
    vals = np.zeros(dm.num_element_dofs(0)*gdim)
    # TODO: check this for parallel operation...
    istart, iend = Dh.getOwnershipRange()
    print ("This process owns the range %d - %d" % (istart, iend))
    for cell_id in range(Vp.mesh().num_cells()):
        rows = dmp.cell_dofs(cell_id)
        rows = rows[(rows>=istart) & (rows < iend)]
        columns = dm.cell_dofs(cell_id)
        dof_coords = coords[columns,:]
        point = dof_coords[0] # Any point in the (closure of the) cell will do.
        e.evaluate_basis_derivatives_all(1, vals, point, dof_coords, 1)
        #print("rows=%s, columns=%s, point=%s, dof_coords=%s, vals=%s" %
        #     (rows, columns, point, dof_coords, vals.round(2)))
        # Reshape [df_1/dx, df_1/dy, df_2/dx, df_2/dy, ...] into
        # [[df_1/dx, df_2/dx, ...],[df_1/dy, df_2/dy, ...]] :
        # NOTE: the copy() is necessary!
        Dh.setValues(rows, columns, vals.reshape((-1, gdim)).T.copy())
    Dh.assemble()
    return Dh

def compute_discrete_gradient(V, Vp, u):
    """ Returns the discrete gradient of u.

    Arguments
    ---------
        V: Source FunctionSpace of type CG1. (redundant...)
        Vp: Target (Vector)FunctionSpace of type DG0.

    Returns
    -------
        A Function in Vp interpolating the gradient of u.
    """
    Dh = discrete_gradient_operator(V, Vp)
    du = Function(Vp)
    petsc_u = as_backend_type(u.vector()).vec()
    petsc_du = as_backend_type(du.vector()).vec()
    
    Dh.mult(petsc_u, petsc_du)
    du.vector().apply('insert')

    return du

In [ ]:
LEFT, RIGHT = -pi, 2*pi
mesh = IntervalMesh(1000, LEFT, RIGHT)
V = FunctionSpace(mesh, "CG", 1)
Vp = FunctionSpace(mesh, "DG", 0)

#Dh = discrete_gradient_operator(V, Vp)
u = interpolate(Expression("sin(x[0])", degree=1), V)
du0 = interpolate(Expression("cos(x[0])", degree=0), Vp)

In [ ]:
%%time
du = compute_discrete_gradient(V, Vp, u)

In [ ]:
print("Error: %e" % (du.vector() - du0.vector()).norm('linf'))
_ = plot(du)

This is faster than the solution with dense matrices but obviously still much slower (a little below 40 times) than building $M^{-1}$ directly because of the hideously slow python loops:

In [ ]:
%%time

dgrad_f = compute_discrete_gradient(W, Q, f)

In [ ]:
print("Error: %e" % (dgrad_f.vector() - grad_f0.vector()).norm('linf'))

# Discrete gradient for DKT

We proceed with the implementation of

$$\nabla_h:W_h \rightarrow \Theta_h$$

for Functions over the whole space. Recall that the operator $\nabla_h$ is given cell-wise by the matrix

\begin{equation}
  M_T = \left(\begin{array}{cccccc}
    0 & I_2 & 0 & 0 & 0 & 0\\
    0 & 0 & 0 & I_2 & 0 & 0\\
    0 & 0 & 0 & 0 & 0 & I_2\\
    0 & 0 & \tilde{t}_{S_1} & \tilde{T}_{S_1} & - \tilde{t}_{S_1} &
    \tilde{T}_{S_1}\\
    \tilde{t}_{S_2} & \tilde{T}_{S_2} & 0 & 0 & - \tilde{t}_{S_2} &
    \tilde{T}_{S_2}\\
    \tilde{t}_{S_3} & \tilde{T}_{S_3} & - \tilde{t}_{S_3} & \tilde{T}_{S_3} &
    0 & 0
  \end{array}\right) \in \mathbb{R}^{12 \times 9}, \quad \tilde{T}_{S_l} =
  \tfrac{- 3}{4} t_{S_l} t_{S_l}^{\top} + \tfrac{1}{2} I_2, 
  \quad \tilde{t}_{S_l} = \tfrac{- 3}{2
  | S_l |} t_{S_l}
\end{equation}

Since part of $M_T$ won't change, we compute it in two steps: an initialization step, which sets the fixed entries and a per-cell step, where we set the entries for each cell.

**NOTE:** Because we are not using a hierarchical basis, the computations in (Bartels, 2015) Ch.8 don't apply verbatim and our $\tilde{T}_{S_l}$ differs by a constant $+ \tfrac{1}{2} I_2$.

In [ ]:
class DKTCellGradient(object):
    """ Computes the transformation matrix for $\nabla_h$ over one cell.
    
    This assumes that the dofs of the "source" cell (in DKT) are ordered as
    
         w1, w1_x, w1_y, w2, w2_x, w2_y, w3, w3_x, w3_y,
         
    i.e. point evaluation, partial derivatives for each vertex in turn, and
    those of the "target" cell (in P_2^2) as
    
        t1_0, t1_1, t2_0, t2_1, t3_0, t3_1,
        ts1_0, ts1_1, ts2_0, ts2_1, ts3_0, ts3_1
        
    i.e. values at the three vertices, then values at the the midpoints of
    the opposite sides.
    """
    def __init__(self):
        """ Initialize the local cell gradient matrix.
        The matrix is 12x9  (target's local space_dimension() x source's 
        local space_dimension()."""

        self.M = np.zeros((12, 9), dtype=np.float)
        rows = [[0,1], [2,3], [4,5], [6,7], [8,9], [10,11]]
        # Hack to make fancy indexing with lists work:
        # if one of rows[i], self.cols[j] has shape 2,1, broadcasting will actually
        # compute the outer product of rows[i] and cols[j], instead of a "zip"
        # of sorts.
        self.rows = list(map(lambda x: np.array(x).reshape(2,1), rows))
        self.cols = [0, [1,2], 3, [4,5], 6, [7,8]]
        #self.cols = list(map(lambda x: np.array(x).reshape((-1,1)), self.cols))
        # TODO: Note that storing these copies of the identity is silly
        # When we apply the operator, it would make more sense to just copy
        # the partial derivative dofs (which is what these first 3 multiplications 
        # with the identity do), then multiply with the lower part of M...
        # (But we still need these here if we insist on building a global matrix
        # which acts on all dofs of the space simultaneously)
        self.M[self.rows[0], self.cols[1]] = np.eye(2)
        self.M[self.rows[1], self.cols[3]] = np.eye(2)
        self.M[self.rows[2], self.cols[5]] = np.eye(2)

        # We are only defined for triangles (to do: assert this)
        # tdim = 2, num_vertices = 3

        # Init temporaries
        self.tt = np.empty((3, 2))
        self.TT = np.empty((3, 2, 2))
        self.ss = np.empty(3)

    def update(self, cell:Cell):
        """Update the mutating entries of M for the given cell."""
        cc = cell.get_vertex_coordinates().reshape(-1, 2)
        self.tt[0] = (cc[2] - cc[1])
        self.tt[1] = -(cc[0] - cc[2])  # Magic: why was this minus sign here again?
        self.tt[2] = (cc[1] - cc[0])
        for i in range(3):
            self.ss[i] = np.linalg.norm(self.tt[i], ord=2)
            self.tt[i] /= self.ss[i]
            self.TT[i] = -3./4 * np.outer(self.tt[i], self.tt[i])
            # HERE! if we don't use the hierarchical basis we need to add this, see...
            self.TT[i] += 0.5*np.eye(2)
            self.tt[i] *= -3./(2*self.ss[i])

        self.M[self.rows[3], self.cols[2]] = self.tt[0].copy().reshape(2,1)
        self.M[self.rows[3], self.cols[3]] = self.TT[0].copy()
        self.M[self.rows[3], self.cols[4]] = -self.tt[0].copy().reshape(2,1)
        self.M[self.rows[3], self.cols[5]] = self.TT[0].copy()

        self.M[self.rows[4], self.cols[0]] = self.tt[1].copy().reshape(2,1)
        self.M[self.rows[4], self.cols[1]] = self.TT[1].copy()
        self.M[self.rows[4], self.cols[4]] = -self.tt[1].copy().reshape(2,1)
        self.M[self.rows[4], self.cols[5]] = self.TT[1].copy()

        self.M[self.rows[5], self.cols[0]] = self.tt[2].copy().reshape(2,1)
        self.M[self.rows[5], self.cols[1]] = self.TT[2].copy()
        self.M[self.rows[5], self.cols[2]] = -self.tt[2].copy().reshape(2,1)
        self.M[self.rows[5], self.cols[3]] = self.TT[2].copy()

We now aggregate the previous cell-wise operator into one acting over discrete functions over the whole space.

**FIXME:** the following code is very inefficient:

1. There is no need to create and multiply by the first 6 rows of the cell matrix. If we didn't create a "global matrix", i.e. acting on whole vectors of coefficients, but instead proceeded cell by cell this would be faster (albeit at the cost of a python loop, which might prove to be even worse)
2. When building such potentially large sparse matrices one should preinit the sparsity pattern.
3. It should be possible to loop through all facets instead of cells. This should reduce computing time because of the fewer updates to the matrix.

In [ ]:
def dkt_gradient_operator(V, Vp):
    """ Build sparse PETSc matrix of the discrete gradient
    operator between V and Vp.
    
    Arguments
    ---------
        V: Source FunctionSpace of type DKT.
        Vp: Target (Vector)FunctionSpace of type P2
            (WITHOUT a hierarchical basis)
        
    Returns
    -------
        Dh: PETSc sparse matrix such that for u in V,
              Dh * u.vector().array()
            yields the vector of coefficients of the discrete
            gradient of u in Vp.
    """   
    gdim = V.mesh().geometry().dim()
    assert V.ufl_element().shortstr()[:3] == 'DKT',\
           "I need a DKT source space"
    assert Vp.ufl_element().shortstr()[:14] == 'Vector<%d x CG2' % gdim,\
           "I need a CG2 VectorFunctionSpace with %d components" % gdim
    assert Vp.mesh().num_cells() == V.mesh().num_cells(),\
           "Number of cells in the meshes don't match"
    dm = V.dofmap()
    dmp1, dmp2 = Vp.sub(0).dofmap(), Vp.sub(1).dofmap()
    e = V.element()
    ep = Vp.element()
    tdim = e.topological_dimension()
    assert tdim == 2, "I need topological dimension 2 (was %d)" % tdim
    coords = V.tabulate_dof_coordinates().reshape((-1, tdim))

    Dh = PETSc.Mat()
    Dh.create(PETSc.COMM_WORLD)
    Dh.setSizes([Vp.dim(), V.dim()])
    Dh.setType("aij")
    Dh.setUp()
    # In order not to assume the following it would be enough
    # to create the array inside the loop...
    warning("Assuming all cells have the same number of dofs")

    ## Preinit element-wise gradient
    grad = DKTCellGradient()

    # TODO: check this for parallel operation...
    istart, iend = Dh.getOwnershipRange()
    #print ("This process owns the range %d - %d" % (istart, iend))

    for cell_id in range(V.mesh().num_cells()):
        cell = Cell(V.mesh(), cell_id)
        grad.update(cell)
        # Massage dofs into the ordering we require with
        # subspace components of each dof consecutive.
        dest_rows = np.array(list(chain.from_iterable(zip(dmp1.cell_dofs(cell_id),
                                                          dmp2.cell_dofs(cell_id)))),
                             dtype=np.intc)
        dest_rows = dest_rows[(dest_rows>=istart) & (dest_rows < iend)]
        # dofs in V are already in the proper ordering (point eval, gradient eval, ...)
        dest_columns = dm.cell_dofs(cell_id)

        # NOTE: the copy() is necessary!
        #print("Setting:\n\trows=%s\n\tcols=%s\nwith:\n%s" % (dest_rows, dest_columns, M))
        Dh.setValues(dest_rows, dest_columns, grad.M.copy())
        assert len(dest_rows)*len(dest_columns) ==  grad.M.size, "duh"
    Dh.assemble()
    return Dh

It remains only to add an interface to the previous computations:

In [ ]:
def compute_dkt_gradient(V, Vp, u):
    """ Returns the discrete gradient of u.

    Arguments
    ---------
        V: Source FunctionSpace of type DKT. (redundant...)
        Vp: Target VectorFunctionSpace of type CG2.

    Returns
    -------
        A Function in Vp interpolating the gradient of u.
    """
    Dh = dkt_gradient_operator(V, Vp)
    du = Function(Vp)
    petsc_u = as_backend_type(u.vector()).vec()
    petsc_du = as_backend_type(du.vector()).vec()
    
    Dh.mult(petsc_u, petsc_du)
    du.vector().apply('insert')

    return du

## Tests

In [ ]:
import nbimporter
from utils import ExpressionAD
from interpolation import interpolate_hermite
import mshr
import autograd as ad

domain = mshr.Rectangle(Point(1.0, 1.0), Point(2.0, 2.0))
mesh = mshr.generate_mesh(domain, 10)
W = FunctionSpace(mesh, 'DKT', 3)
T = VectorFunctionSpace(mesh, 'Lagrange', 2, dim=2)

In [ ]:
def fun(x,y):
    return x*y**2 # 2*x**2*y/ad.numpy.sqrt(y)  # This breaks things
f = ExpressionAD(fun=fun, degree=3, domain=mesh)
w = interpolate_hermite(f, W)

In [ ]:
#%%timeit  #(~24ms)
gf = project(nabla_grad(f), T)

In [ ]:
#%%timeit  #(~237ms)
dgf = compute_dkt_gradient(W, T, w)
_ = plot(w)

In [ ]:
if not np.allclose(gf.vector().array(), dgf.vector().array(), atol=1e-11):
    dgfa, gfa = dgf.vector().array(), gf.vector().array()
    difs = np.abs(dgfa-gfa)/np.maximum(np.abs(dgfa), np.abs(gfa))
    print("Results differ. Maximum difference = %.3e, mean relative difference = %.3e, std dev = %.3e" 
          % (np.max(np.abs(dgfa-gfa)), difs.mean(), difs.std()))
    pl.figure(figsize=(20,10))
    pl.subplot(1,2,1)
    plot(mesh, alpha=0.3)
    plot(gf)
    pl.title("Projection")
    pl.subplot(1,2,2)
    plot(mesh, alpha=0.3)
    plot(dgf)
    _ = pl.title("Discrete operator")
else:
    print("The gradients agree!!!")

Note how values at the vertices seem to be correct, yet there is still a difference. The issue seems to be mostly at the facet dofs: To see this let us first extract both vertex and facet dofs for each subspace, then compute the list of dofs where the projected gradient and the one computed by the discrete operator disagree:

In [ ]:
from dofs import plot_dofs, plot_field_at_dofs, extract_dofs_with_mask

vertex_dofs_0 = extract_dofs_with_mask(T, [0,1,2])   # T.sub(0)
vertex_dofs_1 = extract_dofs_with_mask(T, [6,7,8])   # T.sub(1)
facet_dofs_0 = extract_dofs_with_mask(T, [3,4,5])    # T.sub(0)
facet_dofs_1 = extract_dofs_with_mask(T, [9,10,11])  # T.sub(1)

In [ ]:
def bad_dofs(f:Function, g:Function, dofs0, dofs1, plot=True, eps=1e-6):
    """
        f,g: Functions over the same VectorFunctionSpace V of dim=2
        dofs0,
        dofs1: dofs in V.sub(0), V.sub(1), i.e. they should be of the
               same length and go in pairs"""
    assert f.geometric_dimension() == g.geometric_dimension() == 2, "Need 2d vector valued functions"
    assert f.function_space().element().topological_dimension() == 2, "Need a 2d domain"
    vf = gf.vector().array()
    vg = dgf.vector().array()
    assert len(dofs0) == len(dofs1)
    num_dofs = len(dofs0)
    bad_dofs = []
    for i,j in zip(dofs0, dofs1): # range(num_vertices):
        try:
            dx, dy = vg[i] / vf[i], vg[j] / vf[j]
            if not (near(dx, 0, eps=eps) or near(dx, 1, eps=eps)) or\
               not (near(dy, 0, eps=eps), near(dy, 1, eps=eps)):
                #print("Not colinear with factors = {},{}".format(dx, dy))
                bad_dofs.append(i)
        except Exception as e:
            print("{} Skipping {}".format(e, i))
    print("Computed gradients differ at %.2f%% of the dofs." % 
          (100*len(bad_dofs)/num_dofs))
    if plot:
        plot_dofs(T, bad_dofs, s=60, c='red', alpha=0.6)
        plot_field_at_dofs(f, dofs0, dofs1, alpha=0.7, color='orange')
        plot_field_at_dofs(g, dofs0, dofs1, alpha=0.8, color='green')
    else:
        return bad_dofs

And here are two nice plots. The red dots are placed at dofs where there is a discrepancy between both methods:

In [ ]:
pl.figure(figsize=(24,12))
pl.subplot(1,2,1)
bad_dofs(gf, dgf, vertex_dofs_0, vertex_dofs_1, eps=1e-3)
pl.title("Vertex dofs")
pl.subplot(1,2,2)
bad_dofs(gf, dgf, facet_dofs_0, facet_dofs_1, eps=1e-3)
_ = pl.title("Facet dofs")

It seems that we are basically ok, up to (sometimes gross) numerical errors (which **may build up!**, so we should be careful here...).

## Iterating over facets

As mentioned above, the construction of the global matrix for $\nabla_h$ is extremely inefficient for several reasons. One is that we perform useless multiplications with the identity matrix, another that we go twice over all inner facets.

FINISH THIS if the C++ implementation doesn't pan out.

In [ ]:
from dofs import facets_dofmap

dm = facets_dofmap(T)
dm.facet_dofs[12]

## C++ implementation

The only sensible way to make the code above faster is to ditch python and integrate the computation of $\nabla_h$ into the assembly process. A straightforward approach is to basically copy `dolfin/fem/Assembler.*` and adapt it to our problem.

But first we need to compute the `DKTGradientOperator` local to a cell. Here follows a first naive implementation (lots of copying etc.):

```c++
  /// Computes the transformation matrix for $\nabla_h$ over one cell.    
  /// This assumes that the dofs of the "source" cell (in DKT) are ordered as
  /// 
  ///      w1, w1_x, w1_y, w2, w2_x, w2_y, w3, w3_x, w3_y,
  ///      
  /// i.e. point evaluation, partial derivatives for each vertex in turn, and
  /// those of the "target" cell (in P_2^2) as
  /// 
  ///     t1_0, t1_1, t2_0, t2_1, t3_0, t3_1,
  ///     ts1_0, ts1_1, ts2_0, ts2_1, ts3_0, ts3_1
  ///     
  /// i.e. values at the three vertices, then values at the the midpoints of
  /// the opposite sides.
  class DKTGradient
  {
    std::array<double, 3*2> tt;    // tangent vectors
    std::array<double, 3*2*2> TT;  // tt_i x tt_i^T (tensor prods.)
    std::array<double, 3> ss;      // cell side lengths
    std::array<double, 12*9> M;    // gradient matrix
    std::vector<double> cc;        // cell coordinates

  public:
    DKTGradient()
    {
      /// Initialise the local cell gradient matrix.
      /// (interpolates from P_2^2 into DKT)
      M.fill(0);  // Necessary?

      // Fill identity submatrices
      auto RC = [](size_t r, size_t c) -> size_t { return r*9 + c; };
      M[RC(0,1)] = 1; M[RC(1,2)] = 1;
      M[RC(2,4)] = 1; M[RC(3,5)] = 1;
      M[RC(4,7)] = 1; M[RC(5,8)] = 1;
    }
    
    /// Updates the operator matrix for the given Cell
    void update(const Cell& cell)
    {
      // Vector and matrix access helpers for tt, TT and M
      // respectively. I guess these will be optimized away...
      auto IJ = [](size_t i, size_t j) -> size_t 
          { return i*2 + j; };
      auto IIJ = [](size_t k, size_t i, size_t j) -> size_t 
          { return 4*k + i*2 + j; };
      auto RC = [](size_t r, size_t c) -> size_t 
          { return r*9 + c; };

      cell.get_vertex_coordinates(cc);
      
      // FIXME: shouldn't this depend on the orientation?
      tt[IJ(0,0)] = cc[IJ(2,0)] - cc[IJ(1,0)];
      tt[IJ(0,1)] = cc[IJ(2,1)] - cc[IJ(1,1)];      
      tt[IJ(1,0)] = cc[IJ(2,0)] - cc[IJ(0,0)];  // Why is this negated?
      tt[IJ(1,1)] = cc[IJ(2,1)] - cc[IJ(0,1)];
      tt[IJ(2,0)] = cc[IJ(1,0)] - cc[IJ(0,0)];
      tt[IJ(2,1)] = cc[IJ(1,1)] - cc[IJ(0,1)];
      
      auto outer = [&](double x0, double x1) -> std::array<double, 2*2> {
        return { x0*x0, x0*x1, x1*x0, x1*x1 };
      };
      
      for (int i=0; i < 3; ++i) {
        ss[i] = std::sqrt(tt[IJ(i,0)]*tt[IJ(i,0)] + tt[IJ(i,1)]*tt[IJ(i,1)]);
        tt[IJ(i,0)] /= ss[i];
        tt[IJ(i,1)] /= ss[i];
        auto m = outer(tt[IJ(i,0)], tt[IJ(i,1)]);
        TT[IIJ(i,0,0)] = 0.5 - 0.75*m[IJ(0,0)];
        TT[IIJ(i,0,1)] =     - 0.75*m[IJ(0,1)];
        TT[IIJ(i,1,0)] =     - 0.75*m[IJ(1,0)];
        TT[IIJ(i,1,1)] = 0.5 - 0.75*m[IJ(1,1)];
        tt[IJ(i,0)] *= -3/(2*ss[i]);
      }

      // Copy onto the gradient (sub) matrix 
      // This is quite wasteful...
      auto copytt = [&](size_t i, size_t r, size_t c) {
        M[RC(r,c)] = tt[IJ(i,0)]; M[RC(r+1,c)] = tt[IJ(i,1)];
      };
      auto copy_tt = [&](size_t i, size_t r, size_t c) {
        M[RC(r,c)] = -tt[IJ(i,0)]; M[RC(r+1,c)] = -tt[IJ(i,1)];
      };
      auto copyTT = [&](size_t i, size_t r, size_t c) {
        M[RC(r,c)] = TT[IIJ(i,0,0)]; M[RC(r,c+1)] = TT[IIJ(i,0,1)];
        M[RC(r+1,c)] = TT[IIJ(i,1,0)]; M[RC(r+1,c+1)] = TT[IIJ(i,1,1)];
      };

      copytt(0, 3, 2);
      copyTT(0, 3, 3);
      copy_tt(0, 3, 4);
      copyTT(0, 3, 5);
      
      copytt(0, 4, 0);
      copyTT(0, 4, 1);
      copy_tt(0, 4, 4);
      copyTT(0, 4, 5);
      
      copytt(0, 5, 0);
      copyTT(0, 5, 1);
      copy_tt(0, 5, 2);
      copyTT(0, 5, 3);
    }

    /// Transform local dof coefficients from P_3^red to P_2^2
    void apply(const std::vector<double>& p3coeffs, 
               std::vector<double>& p22coeffs)
    {
      // Copy coefficients for partial derivatives
      p22coeffs[0] = p3coeffs[1];  p22coeffs[1] = p3coeffs[2];
      p22coeffs[2] = p3coeffs[4];  p22coeffs[3] = p3coeffs[5];
      p22coeffs[4] = p3coeffs[7];  p22coeffs[5] = p3coeffs[8];

      // Matrix * vec multiplication
      for (auto i = 0; i < 6; ++i) {
        p22coeffs[6+i] = 0;
        for (auto j = 0; j < 9; ++j) {
           p22coeffs[6+i] += M[i*9+j]*p3coeffs[j];
        }
      }
    }
  }
```

Once that works we use it in the assembler. We need to be careful with:

1. Combining the dofmaps from two forms: this won't be that hard. We compute the local tensor for the P2 form then compute $M^T A M$ and apply the local-to-global dofmap for the DKT space in order to populate the global tensor.

2. Precomputing the sparsity pattern. This is done in `AssemblerBase::init_global_tensor(GenericTensor& A, const Form& a)` and relies on the FunctionSpaces associated to the Form being assembled, so we'd need to rewrite this.

The resulting assembler is available in `KirchhoffAssembler.cpp`, which is basically a copy of `dolfin/fem/Assembler.cpp` with modifications to call the routines in `DKTGradient` at the relevant places.

## Tests

Ideally we'd like to make the assembler available from the python interface. However this involves a bit of boilerplate, so I'll leave the following stub here in case I decide to come back to it in the future.

In [ ]:
from dolfin import *
import numpy
import os

src_dir = "/home/fenics/local/src/nonlinear-kirchhoff/"
typemap ="""
%typemap(in) (const std::vector<double>& cc) {
    if (! PyArray_Check($input)) {
        PyErr_SetString(PyExc_TypeError, "Not a NumPy array");
        return NULL;
    }
    PyArrayObject* pyarray;
    pyarray = (PyArrayObject*)$input;
    if (!(PyArray_TYPE(pyarray) == NPY_DOUBLE)) {
      PyErr_SetString(PyExc_TypeError, "Not a NumPy array of doubles");
      return NULL;
    }
    auto vec = std::vector<double>(pyarray->dimensions[0]);
    for(int i = 0; i < pyarray->dimensions[0]; ++i)
        vec[i] = pyarray->data[i];
    $1 = vec;
}
"""
with open(src_dir + "DKTGradient.h", "r") as header_file:
    code = header_file.read()
    dkg = compile_extension_module(code=code,
                                   additional_declarations=typemap,
                                   source_directory=src_dir,
                                   sources=["DKTGradient.cpp"], 
                                   include_dirs=[".", os.path.abspath(src_dir), "/usr/local/slepc-64/include/"])

In [ ]:
W = VectorFunctionSpace(UnitSquareMesh(10,10, "right"), "DKT", degree=3, dim=3)
c = Cell(W.mesh(), 0)

dg = dkg.DKTGradient
dg.update(c)

# Dump

Everything and anything...

In [ ]:
import nbimporter
from dofs import plot_dofs
%matplotlib inline

In [ ]:
v2d = vertex_to_dof_map(W3)
print(v2d)
for vit in range(msh.num_vertices()):
    v = Vertex(msh, vit)
    for sub in range(3):
        print(v2d[9*v.index()+3*sub+1])
plot_dofs(W3, [15])

In [ ]:
msh = UnitSquareMesh(1,1)
W3 = VectorFunctionSpace(msh, "DKT", 3, dim=3)
T6 = TensorFunctionSpace(msh, "Lagrange", 2, shape=(3,2))
dmW = W3.sub(0).dofmap()
c = Cell(msh, 0)
print(c.get_coordinate_dofs())
dofs = dmW.cell_dofs(0)
print(dofs)
# local dofs for the P2 element are eval at v0,v1,v2, then eval at s1,s2,s3:
#plot_dofs(W3, dofs[dmW.tabulate_entity_dofs(0,2)])
#plot_dofs(W3, [6])

In [ ]:
dmT = T6.sub(1).dofmap()
dmT.cell_dofs(0)

In [ ]:
from dolfin import *
import nbimporter
from discrete_gradient import DKTCellGradient
import numpy as np

#ACHTUNG!
np.set_printoptions(precision=4, linewidth=120)

def cppprint(x, prec=4):
    m, n = x.shape
    for i in range(m):
        for j in range(n):
            print(str(np.round(x[i,j], prec)) + ", ", end='')
        print("")

domain = UnitSquareMesh(1,1)
V = VectorFunctionSpace(domain, "Lagrange", dim=2, degree=2)
u = TrialFunction(V)
v = TestFunction(V)

a = inner(nabla_grad(u), nabla_grad(v))*dx
A = assemble(a)

grad = DKTCellGradient()
Aa = A.array()
A2 = np.zeros((12,12))
dm = V.dofmap()
for cell_id in range(V.mesh().num_cells()):
    cell = Cell(V.mesh(), cell_id)
    print(cell.get_coordinate_dofs())
    grad.update(cell)
    print("***** M ******")
    cppprint(grad.M)
    print("***** A2 ******")
    l2g = dm.cell_dofs(cell_id)   # local to global mapping for cell 
    for i,j in np.ndindex(A2.shape):
        A2[i,j] = Aa[l2g[i], l2g[j]]
    cppprint(A2)
    print("============= Mt @ A2 @ M ================")
    cppprint(grad.M.transpose() @ A2 @ grad.M)

## A quick retesting of the assembly of the matrix for $\nabla_h$

In [ ]:
mesh = UnitSquareMesh(1,1)
W = FunctionSpace(mesh, 'DKT', 3)
T = VectorFunctionSpace(mesh, 'Lagrange', 2, dim=2)

w = Function(W)
dmw = W.dofmap()
wdofs_cell0 = dmw.cell_dofs(0)
wdofs_cell1 = dmw.cell_dofs(1)
new_vals = np.zeros_like(w.vector().array())
new_vals[wdofs_cell0] = [2,1,1,2,1,1,2,1,1]
w.vector().set_local(new_vals)


Dh = DKTCellGradient(T.element().space_dimension(), W.element().space_dimension())
Dh.update(Cell(W.mesh(), 0))
product = Dh.M@w.vector().array()[wdofs_cell0]

dw2 = Function(T)
dm0 = T.sub(0).dofmap()
dm1 = T.sub(1).dofmap()
tdofs0_cell0 = dm0.cell_dofs(0)
tdofs1_cell0 = dm1.cell_dofs(0)
#### <strike>DON'T FIXME!!!! this is NOT WRONG:</strike>
dest_rows = np.array(list(chain.from_iterable(zip(tdofs0_cell0,tdofs1_cell0))), dtype=np.intc)
print(tdofs0_cell0, tdofs1_cell0)
print(dest_rows)
#### found manually:
dest_rows = [2, 3, 12, 13, 4, 5, 14, 15, 6, 7, 16, 17]
new_vals2 = np.zeros_like(dw2.vector().array())
new_vals2[dest_rows] = product
dw2.vector().set_local(new_vals2)
print(product)
print(Dh.M)

pl.figure(figsize=(12,6))
plot(T.mesh(), alpha=0.3)
vertex_dofs_0 = extract_dofs_with_mask(T, [0,1,2])   # T.sub(0)
vertex_dofs_1 = extract_dofs_with_mask(T, [6,7,8])   # T.sub(1)
facet_dofs_0 = extract_dofs_with_mask(T, [3,4,5])    # T.sub(0)
facet_dofs_1 = extract_dofs_with_mask(T, [9,10,11])  # T.sub(1)
plot_field_at_dofs(dw2, vertex_dofs_0, vertex_dofs_1, color='orange')
plot_field_at_dofs(dw2, facet_dofs_0, facet_dofs_1, color='green')